In [ ]:
"""
accuracy with NN 0.7956  0.7881
accuracy with GA 0.8305
accuracy with fS 76.28   77.97
accuracy with GFS 76.64% 77.97%
accuracy with FN  0.7883   0.8559
accuracy with GFFN

"""

In [38]:
import pandas as pd

# Read the dataset
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [22]:
!pip install scikit-fuzzy
!pip install deap

In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from deap import base, creator, tools, algorithms
import random

In [39]:
data = data[
    (data['Glucose'] != 0) &
    (data['BloodPressure'] != 0) &
    (data['SkinThickness'] != 0) &
    (data['Insulin'] != 0) &
    (data['BMI'] != 0)
]

In [25]:
class_distribution = data['Outcome'].value_counts()

# Display the class distribution
print(class_distribution)

0    262
1    130
Name: Outcome, dtype: int64


In [49]:
from sklearn.model_selection import StratifiedShuffleSplit

x = data.iloc[:,0:8]
y = data.iloc[:,8]
# Assuming 'X' is your feature matrix and 'y' is your target variable
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(x, y):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Print the class distribution in the training and testing sets
print("Training set class distribution:\n", y_train.value_counts())
print("\nTesting set class distribution:\n", y_test.value_counts())

Training set class distribution:
 0    183
1     91
Name: Outcome, dtype: int64

Testing set class distribution:
 0    79
1    39
Name: Outcome, dtype: int64


In [50]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()
data_scaled = scaler.fit_transform(x_train)

x_train_combined = pd.DataFrame(data_scaled, columns=x_train.columns)
x_train_combined.head()

data_scaledd = scaler.fit_transform(x_test)

x_test_combined = pd.DataFrame(data_scaledd, columns=x_test.columns)

In [51]:
classifier_combined = MLPClassifier(hidden_layer_sizes=(8, 4), max_iter=1000, random_state=1,alpha=10)
classifier_combined.fit(x_train_combined, y_train)
y_pred_combined_a = classifier_combined.predict(x_train_combined)
accuracy_combined_a = accuracy_score(y_train, y_pred_combined_a)
print("Accuracy of the Combined Model:", accuracy_combined_a)
    # Make predictions using the trained neural network
y_pred_combined = classifier_combined.predict(x_test_combined)

    # Evaluate the performance of the combined model
accuracy_combined = accuracy_score(y_test, y_pred_combined)
print("Accuracy of the Combined Model:", accuracy_combined)

Accuracy of the Combined Model: 0.7956204379562044
Accuracy of the Combined Model: 0.788135593220339


In [29]:
# Step 4: Implement Genetic Feature Selection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
#import skfuzzy as fuzz
#from skfuzzy import control as ctrl
from deap import base, creator, tools, algorithms
import random



# Step 4: Implement Genetic Feature Selection

# Define the evaluation function for the genetic algorithm
def evaluate(features):
    selected_features = [i for i in range(len(features)) if features[i] == 1]
    x_train_subset = x_train_combined.iloc[:, selected_features]
    x_test_subset = x_test_combined.iloc[:, selected_features]

    # Train and evaluate the classifier (you can use your own GNFS model)
    classifier = MLPClassifier(hidden_layer_sizes=(8, 4), max_iter=10000, random_state=1,alpha=11)
    classifier.fit(x_train_subset, y_train)
    y_pred = classifier.predict(x_test_subset)
    return accuracy_score(y_test, y_pred),

# Define genetic algorithm parameters and functions
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=8)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    population_size = 20
    generations = 6

    # Create an initial population of feature subsets
    population = toolbox.population(n=population_size)

        # Initialize a variable to store the best individual found so far
    best_individual = None
    best_accuracy = 0.0  # Initialize with a low value

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit



    # Perform the genetic algorithm
    for gen in range(generations):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
        fitnesses = list(map(toolbox.evaluate, offspring))
        for ind, fit in zip(offspring, fitnesses):
            ind.fitness.values = fit

        population = toolbox.select(offspring, k=len(population))

    # Get the best individual (feature subset)
    best_individual = tools.selBest(population, k=1)[0]
    selected_features = [i for i in range(len(best_individual)) if best_individual[i] == 1]

    # Calculate accuracy for the best individual
    best_accuracy = evaluate(best_individual)[0]

    print("Best selected features:", selected_features)
    print("Accuracy of the best individual:", best_accuracy)
    print("Accuracy ll:", evaluate(selected_features))
    print(best_individual)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Best selected features: [0, 1, 4, 5, 6, 7]
Accuracy of the best individual: 0.8305084745762712
Accuracy ll: (0.7288135593220338,)
[1, 1, 0, 0, 1, 1, 1, 1]


In [30]:
#Define fuzzy sets

def pregnanciesT(pregnancies:ctrl.Antecedent):
  pregnancies['low'] = fuzz.trimf(pregnancies.universe,[0, 0.475, 7.213])
  pregnancies['medium'] = fuzz.trimf(pregnancies.universe,[0.475, 7.213, 12.1])
  pregnancies['high'] = fuzz.trimf(pregnancies.universe,[7.213, 12.1, 17])

def glucoseT(glucose: ctrl.Antecedent):
    glucose['low'] = fuzz.trimf(glucose.universe, [0, 88.84, 152.6])
    glucose['medium'] = fuzz.trimf(glucose.universe, [88.84, 152.6, 175.8])
    glucose['high'] = fuzz.trimf(glucose.universe, [152.6, 175.8, 199])

def bloodPressureT(bloodPressure: ctrl.Antecedent):
    bloodPressure['low']=fuzz.trimf(bloodPressure.universe,[0,49.73,88.44])
    bloodPressure['medium'] =fuzz.trimf(bloodPressure.universe,[49.73,88.44,105.2])
    bloodPressure['high'] =fuzz.trimf(bloodPressure.universe,[88.44,105.2,122])

def skinThicknessT(skinThickness: ctrl.Antecedent):
    skinThickness['good'] =fuzz.trimf(skinThickness.universe,[0,4.584,36.49])
    skinThickness['average'] =fuzz.trimf(skinThickness.universe,[4.584,36.49,67.74])
    skinThickness['below'] =fuzz.trimf(skinThickness.universe,[36.49,67.74,99])

def insulinT(insulin: ctrl.Antecedent):
    insulin['low'] =fuzz.trimf(insulin.universe,[0,35.44,195])
    insulin['medium'] =fuzz.trimf(insulin.universe,[35.44,195,520.5])
    insulin['high'] =fuzz.trimf(insulin.universe,[195,520.5,846])

def bmiT(bmi: ctrl.Antecedent):
    bmi['low'] =fuzz.trimf(bmi.universe,[0,24.11,39.87])
    bmi['medium'] =fuzz.trimf(bmi.universe,[24.11,39.87,53.49])
    bmi['high'] =fuzz.trimf(bmi.universe,[39.87,53.49,67.1])

def pedigreeT(pedigree: ctrl.Antecedent):
    pedigree['low'] =fuzz.trimf(pedigree.universe,[0.078,0.1405,0.8032])
    pedigree['medium'] =fuzz.trimf(pedigree.universe,[0.1405,0.8032,1.612])
    pedigree['high'] =fuzz.trimf(pedigree.universe,[0.8032,1.612,2.42])

def ageT(age: ctrl.Antecedent):
    age['low'] =fuzz.trimf(age.universe,[21,21.48,45])
    age['medium'] =fuzz.trimf(age.universe,[21.48,45,63])
    age['high'] =fuzz.trimf(age.universe,[45,63,81])

def outcomeT(outcome: ctrl.Antecedent):
    outcome['low'] =fuzz.trimf(outcome.universe,[0,0.25,0.45])
    outcome['medium'] =fuzz.trimf(outcome.universe,[0.35,0.55,0.75])
    outcome['high'] =fuzz.trimf(outcome.universe,[0.65,0.85,1])


In [31]:
def buildRules(

    pregnancies:ctrl.Antecedent,
    glucose: ctrl.Antecedent,
    bloodPressure: ctrl.Antecedent,
    skinThickness: ctrl.Antecedent,
    insulin: ctrl.Antecedent,
    bmi: ctrl.Antecedent,
    pedigree: ctrl.Antecedent,
    age: ctrl.Antecedent,
    outcome: ctrl.Antecedent,
):
    return [
        ctrl.Rule(pregnancies['low'] | glucose['low'] | skinThickness['good'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),

        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),

        ctrl.Rule(pregnancies['high'] | glucose['high'] | bloodPressure['high'] | skinThickness['below'] | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['high']),


        ctrl.Rule(pregnancies['high'] | glucose['high'] | bloodPressure['medium'] | skinThickness['below'] | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['medium']),


         ctrl.Rule(pregnancies['high'] | glucose['medium'] | bloodPressure['high'] | skinThickness['below'] | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['medium']),

        ctrl.Rule(pregnancies['low'] | glucose['high'] | bloodPressure['high'] | skinThickness['good'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),



        ctrl.Rule(pregnancies['low'] | glucose['high'] | bloodPressure['medium'] | skinThickness['good'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),


        ctrl.Rule(pregnancies['low'] | glucose['medium'] | bloodPressure['high'] | skinThickness['good'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),


        ctrl.Rule(pregnancies['medium'] | glucose['high'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['high'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['high'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['below'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['high']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['high'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['high'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | bloodPressure['medium'] | skinThickness['average'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['high']
                  ,outcome['medium']),


    ]

In [14]:
# Create fuzzy inference system

pregnancies = ctrl.Antecedent(np.arange(1, 17+1, 1), 'pregnancies')
glucose = ctrl.Antecedent(np.arange(0, 199+1, 1), 'glucose')
bloodPressure = ctrl.Antecedent(np.arange(0, 122+1, 1), 'bloodPressure')
skinThickness	= ctrl.Antecedent(np.arange(0, 99+1, 1), 'skinThickness')
insulin = ctrl.Antecedent(np.arange(0, 846+1, 1), 'insulin')
bmi = ctrl.Antecedent(np.arange(0, 67.1+0.1, 0.1), 'bmi')
pedigree = ctrl.Antecedent(np.arange(0.078, 2.42+0.001, 0.001), 'pedigree')
age = ctrl.Antecedent(np.arange(21, 81+0.01, 0.01), 'age')
outcome = ctrl.Consequent(np.arange(0, 1+0.01, 0.01), 'outcome',
                             defuzzify_method="centroid")

# Membership Functions
pregnanciesT(pregnancies = pregnancies)
glucoseT(glucose=glucose)
bloodPressureT(bloodPressure = bloodPressure)
skinThicknessT(skinThickness = skinThickness)
insulinT(insulin = insulin)
bmiT(bmi=bmi)
pedigreeT(pedigree = pedigree)
ageT(age=age)
outcomeT(outcome=outcome)

# Rules
rules = buildRules(
    pregnancies = pregnancies,
    glucose = glucose,
    bloodPressure = bloodPressure,
    skinThickness = skinThickness,
    insulin = insulin,
    age=age,
    bmi=bmi,
    pedigree = pedigree,
    outcome = outcome,
)

# Mamdani control system
prediction_ctrl = ctrl.ControlSystem(rules)
prediction_inference = ctrl.ControlSystemSimulation(prediction_ctrl)


#
prediction_inference.input['pregnancies'] = x_train['Pregnancies'].to_numpy()
prediction_inference.input['glucose'] = x_train['Glucose'].to_numpy()
prediction_inference.input['bloodPressure'] = x_train['BloodPressure'].to_numpy()
prediction_inference.input['insulin'] = x_train['Insulin'].to_numpy()
prediction_inference.input['skinThickness']= x_train['SkinThickness'].to_numpy()
prediction_inference.input['pedigree']= x_train['DiabetesPedigreeFunction'].to_numpy()
prediction_inference.input['age'] = x_train['Age'].to_numpy()
prediction_inference.input['bmi'] = x_train['BMI'].to_numpy()


prediction_inference.compute()
result_np = prediction_inference.output['outcome']

output_np = np.where(result_np > 0.481, 1, 0)
gt_np = y_train.to_numpy()
total_sample = gt_np.shape[0]
accurate_sample = (output_np == gt_np).sum()
accuracy = (accurate_sample / total_sample) * 100
print(total_sample)
print(accurate_sample)
print(f"{accuracy:.2f}%")

274
209
76.28%


In [16]:
prediction_inference.input['pregnancies'] = x_test['Pregnancies'].to_numpy()
prediction_inference.input['glucose'] = x_test['Glucose'].to_numpy()
prediction_inference.input['bloodPressure'] = x_test['BloodPressure'].to_numpy()
prediction_inference.input['insulin'] = x_test['Insulin'].to_numpy()
prediction_inference.input['skinThickness']= x_test['SkinThickness'].to_numpy()
prediction_inference.input['pedigree']= x_test['DiabetesPedigreeFunction'].to_numpy()
prediction_inference.input['age'] = x_test['Age'].to_numpy()
prediction_inference.input['bmi'] = x_test['BMI'].to_numpy()


prediction_inference.compute()
result_np_test = prediction_inference.output['outcome']

output_np_t = np.where(result_np_test > 0.48, 1, 0)
gt_np = y_test.to_numpy()
total_sample = gt_np.shape[0]
accurate_sample = (output_np_t == gt_np).sum()
accuracy = (accurate_sample / total_sample) * 100
print(total_sample)
print(accurate_sample)
print(f"{accuracy:.2f}%")


118
92
77.97%


/usr/local/lib/python3.10/dist-packages/skfuzzy/control/controlsystem.py:178: UserWarning: Input array is shape (118,), which is different from previous array(s) which were (274,).  This may cause problems, unless you are replacing all inputs.
  warn("Input array is shape {0}, which is different from "


In [32]:
def buildRules(

    pregnancies:ctrl.Antecedent,
    glucose: ctrl.Antecedent,
    insulin: ctrl.Antecedent,
    bmi: ctrl.Antecedent,
    pedigree: ctrl.Antecedent,
    age: ctrl.Antecedent,
    outcome: ctrl.Antecedent,
):
    return [
        ctrl.Rule(pregnancies['low'] | glucose['low']| insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),

        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),

        ctrl.Rule(pregnancies['high'] | glucose['high'] | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['high']),


        ctrl.Rule(pregnancies['high'] | glucose['high'] | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['medium']),


         ctrl.Rule(pregnancies['high'] | glucose['medium']  | insulin['high'] | bmi['high']
                  | pedigree['high'] | age['high']
                  ,outcome['medium']),

        ctrl.Rule(pregnancies['low'] | glucose['high'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),



        ctrl.Rule(pregnancies['low'] | glucose['high'] | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),


        ctrl.Rule(pregnancies['low'] | glucose['medium']  | insulin['low'] | bmi['low']
                  | pedigree['low'] | age['low']
                  ,outcome['low']),


        ctrl.Rule(pregnancies['medium'] | glucose['high'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['high'] | glucose['medium']  | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium']  | insulin['high']
                  | bmi['medium'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium']  | insulin['medium']
                  | bmi['high'] | pedigree['medium'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium']  | insulin['medium']
                  | bmi['medium'] | pedigree['high'] | age['medium']
                  ,outcome['medium']),


        ctrl.Rule(pregnancies['medium'] | glucose['medium'] | insulin['medium']
                  | bmi['medium'] | pedigree['medium'] | age['high']
                  ,outcome['medium']),


    ]

In [33]:
# Create fuzzy inference system

pregnancies = ctrl.Antecedent(np.arange(1, 17+1, 1), 'pregnancies')
glucose = ctrl.Antecedent(np.arange(0, 199+1, 1), 'glucose')
bloodPressure = ctrl.Antecedent(np.arange(0, 122+1, 1), 'bloodPressure')
skinThickness	= ctrl.Antecedent(np.arange(0, 99+1, 1), 'skinThickness')
insulin = ctrl.Antecedent(np.arange(0, 846+1, 1), 'insulin')
bmi = ctrl.Antecedent(np.arange(0, 67.1+0.1, 0.1), 'bmi')
pedigree = ctrl.Antecedent(np.arange(0.078, 2.42+0.001, 0.001), 'pedigree')
age = ctrl.Antecedent(np.arange(21, 81+0.01, 0.01), 'age')
outcome = ctrl.Consequent(np.arange(0, 1+0.01, 0.01), 'outcome',
                             defuzzify_method="centroid")

# Membership Functions
pregnanciesT(pregnancies = pregnancies)
glucoseT(glucose=glucose)
bloodPressureT(bloodPressure = bloodPressure)
skinThicknessT(skinThickness = skinThickness)
insulinT(insulin = insulin)
bmiT(bmi=bmi)
pedigreeT(pedigree = pedigree)
ageT(age=age)
outcomeT(outcome=outcome)

# Rules
rules = buildRules(
    pregnancies = pregnancies,
    glucose = glucose,
    #bloodPressure = bloodPressure,
    #skinThickness = skinThickness,
    insulin = insulin,
    age=age,
    bmi=bmi,
    pedigree = pedigree,
    outcome = outcome,
)

# Mamdani control system
prediction_ctrl = ctrl.ControlSystem(rules)
prediction_inference = ctrl.ControlSystemSimulation(prediction_ctrl)


#
prediction_inference.input['pregnancies'] = x_train['Pregnancies'].to_numpy()
prediction_inference.input['glucose'] = x_train['Glucose'].to_numpy()
#prediction_inference.input['bloodPressure'] = x_train['BloodPressure'].to_numpy()
prediction_inference.input['insulin'] = x_train['Insulin'].to_numpy()
#prediction_inference.input['skinThickness']= x_train['SkinThickness'].to_numpy()
prediction_inference.input['pedigree']= x_train['DiabetesPedigreeFunction'].to_numpy()
prediction_inference.input['age'] = x_train['Age'].to_numpy()
prediction_inference.input['bmi'] = x_train['BMI'].to_numpy()


prediction_inference.compute()
result_np = prediction_inference.output['outcome']

output_np = np.where(result_np > 0.481, 1, 0)
gt_np = y_train.to_numpy()
total_sample = gt_np.shape[0]
accurate_sample = (output_np == gt_np).sum()
accuracy = (accurate_sample / total_sample) * 100
print(total_sample)
print(accurate_sample)
print(f"{accuracy:.2f}%")

prediction_inference.input['pregnancies'] = x_test['Pregnancies'].to_numpy()
prediction_inference.input['glucose'] = x_test['Glucose'].to_numpy()
#prediction_inference.input['bloodPressure'] = x_test['BloodPressure'].to_numpy()
prediction_inference.input['insulin'] = x_test['Insulin'].to_numpy()
#prediction_inference.input['skinThickness']= x_test['SkinThickness'].to_numpy()
prediction_inference.input['pedigree']= x_test['DiabetesPedigreeFunction'].to_numpy()
prediction_inference.input['age'] = x_test['Age'].to_numpy()
prediction_inference.input['bmi'] = x_test['BMI'].to_numpy()


prediction_inference.compute()
result_np_test = prediction_inference.output['outcome']

output_np_t = np.where(result_np_test > 0.48, 1, 0)
gt_np = y_test.to_numpy()
total_sample = gt_np.shape[0]
accurate_sample = (output_np_t == gt_np).sum()
accuracy = (accurate_sample / total_sample) * 100
print(total_sample)
print(accurate_sample)
print(f"{accuracy:.2f}%")


274
210
76.64%
118
92
77.97%


/usr/local/lib/python3.10/dist-packages/skfuzzy/control/controlsystem.py:178: UserWarning: Input array is shape (118,), which is different from previous array(s) which were (274,).  This may cause problems, unless you are replacing all inputs.
  warn("Input array is shape {0}, which is different from "


In [34]:
 # Convert 'fuzzy_output' to a DataFrame
fuzzy_features = pd.DataFrame(output_np, columns=['fuzzy_output'])

x_train.reset_index(drop=True, inplace=True)
fuzzy_features.reset_index(drop=True, inplace=True)

# Combine fuzzy output with the scaled training data
x_train_combined = pd.concat([x_train, fuzzy_features], axis=1)


fuzzy_features_t = pd.DataFrame(output_np_t, columns=['fuzzy_output'])
x_test.reset_index(drop=True, inplace=True)
fuzzy_features_t.reset_index(drop=True, inplace=True)

    # Combine fuzzy output with the scaled test data
x_test_combined = pd.concat([x_test, fuzzy_features_t ], axis=1)

In [35]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()
data_scaled = scaler.fit_transform(x_train_combined)

x_train_combined = pd.DataFrame(data_scaled, columns=x_train_combined.columns)
x_train_combined.head()

data_scaledd = scaler.fit_transform(x_test_combined)

x_test_combined = pd.DataFrame(data_scaledd, columns=x_test_combined.columns)
x_test_combined.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,fuzzy_output
0,0.513979,1.058735,1.051660,1.298798,0.409668,0.924832,-0.328550,-0.147378,-0.596927
1,-0.443645,0.528972,-0.940002,0.556371,-0.296666,-1.056009,0.529031,-0.649997,-0.596927
2,-0.124437,-0.530556,-0.633593,-1.670911,-0.985773,-1.409730,0.469790,-0.750520,-0.596927
3,-0.443645,-0.499393,-0.633593,-1.989094,0.995409,-1.070157,1.042451,-0.851044,-0.596927
4,0.513979,0.466646,0.745250,0.662432,-0.021024,-0.178779,-0.424463,-0.549473,-0.596927


In [36]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score

# Fine-Tune Regularization Parameter
for alpha in [0.001, 0.01, 0.1, 1,2,3,4,5,6,7,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20]:
    classifier_combined_regularized = MLPClassifier(hidden_layer_sizes=(8, 4), max_iter=10000, random_state=1, alpha=alpha)
    classifier_combined_regularized.fit(x_train_combined, y_train)
    y_pred_combined = classifier_combined_regularized.predict(x_test_combined)
    accuracy_combined = accuracy_score(y_test, y_pred_combined)
    print(f"Accuracy with alpha={alpha}: {accuracy_combined}")

# Evaluate Additional Metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred_combined))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_combined))

# Cross-Validation
cv_scores = cross_val_score(classifier_combined_regularized, x_train_combined, y_train, cv=5)
print("\nCross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", np.mean(cv_scores))

Accuracy with alpha=0.001: 0.8050847457627118
Accuracy with alpha=0.01: 0.8050847457627118
Accuracy with alpha=0.1: 0.8135593220338984
Accuracy with alpha=1: 0.7627118644067796
Accuracy with alpha=2: 0.8050847457627118
Accuracy with alpha=3: 0.8305084745762712
Accuracy with alpha=4: 0.8220338983050848
Accuracy with alpha=5: 0.8135593220338984
Accuracy with alpha=6: 0.8050847457627118
Accuracy with alpha=7: 0.8135593220338984
Accuracy with alpha=8: 0.8389830508474576
Accuracy with alpha=9: 0.847457627118644
Accuracy with alpha=10: 0.8559322033898306
Accuracy with alpha=11: 0.847457627118644
Accuracy with alpha=12: 0.8220338983050848
Accuracy with alpha=13: 0.8220338983050848
Accuracy with alpha=14: 0.788135593220339
Accuracy with alpha=15: 0.7966101694915254
Accuracy with alpha=16: 0.7711864406779662
Accuracy with alpha=17: 0.7627118644067796
Accuracy with alpha=18: 0.7796610169491526
Accuracy with alpha=19: 0.7627118644067796
Accuracy with alpha=20: 0.7033898305084746

Classification R

FN (all rules)

In [37]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Train a neural network using the combined dataset with regularization
classifier_combined_regularized = MLPClassifier(hidden_layer_sizes=(8, 4), max_iter=10000, random_state=1, alpha=10)
classifier_combined_regularized.fit(x_train_combined, y_train)

# Make predictions on the training set
y_pred_combined_a = classifier_combined_regularized.predict(x_train_combined)
accuracy_combined_a = accuracy_score(y_train, y_pred_combined_a)
print("Accuracy of the Combined Model on Training Set:", accuracy_combined_a)

# Make predictions on the test set
y_pred_combined = classifier_combined_regularized.predict(x_test_combined)

# Evaluate the performance of the combined model on the test set
accuracy_combined = accuracy_score(y_test, y_pred_combined)
print("Accuracy of the Combined Model on Test Set:", accuracy_combined)

Accuracy of the Combined Model on Training Set: 0.781021897810219
Accuracy of the Combined Model on Test Set: 0.8559322033898306
